# Spectral Clustering

## Important imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from scipy.spatial import distance_matrix as distanceMatrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score

## Data and Preprocessing

In [2]:
def loadData(path):
    data = pd.read_csv(path)
    x = data.iloc[:, :-1].values
    y = data.iloc[:, -1].values
    return x, y

def splitData(x, y, train_size):
    x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=train_size, random_state=0, stratify=y)
    return x_train, x_test, y_train, y_test

def preprocessData(x):

    ct = ColumnTransformer( [('one_hot_encoder', OneHotEncoder(), [1, 2, 3])], remainder='passthrough' )
    ct = ct.fit(x)
    x = pd.DataFrame(ct.transform(x))
    # Drop the first, second and third columns
    x = x.drop([1, 2, 3], axis=1)
    return x

## Algorithm

In [3]:
def constructWeightMatrix(data, gamma = 10):
    # Computing weight matrix using rbf kernel
    weight_matrix = np.exp(-gamma * pairwise_distances(data, metric='euclidean') ** 2)
    return weight_matrix

def clusteringUsingNormalizedCut(data, k, gamma = 10, y = None):

    # Constructing the weight matrix
    weight_matrix = constructWeightMatrix(data, gamma=gamma)

    # Computing the degree matrix
    degree_matrix = np.diag(np.sum(weight_matrix, axis=1))

    # Computing the laplacian matrix
    laplacian_matrix = degree_matrix - weight_matrix

    # Computing the eigenvalues and eigenvectors of the laplacian matrix
    eigenvalues, eigenvectors = np.linalg.eig(np.linalg.inv(degree_matrix).dot(laplacian_matrix))
    
    # Taking the only first k eigenvectors
    eigenvectors = eigenvectors[:, :k]

    # Taking only the real part of the eigenvectors
    eigenvectors = np.real(eigenvectors)

    # Convert the eigenvectors to pd dataframe
    eigenvectors = pd.DataFrame(eigenvectors)
    
    # Normalizing the eigenvectors
    eigenvectors = eigenvectors.apply(lambda x: (x - np.mean(x)) / np.std(x), axis=0)
    
    # Applying k-means on the eigenvectors
    kmeans = KMeans(n_clusters=k, random_state=0).fit(eigenvectors)

    # Getting the labels of the points
    labels = kmeans.labels_ 

    # Create a list of clusters of size k
    cluster = [[] for i in range(k)]

    # Assign each point to its cluster
    for i in range(len(labels)):
        cluster[labels[i]].append(i)
    
    # Create array name clusterLabels to store the labels of the clusters based on the most frequent label in the cluster in y
    clusterLabels = []
    for i in range (k):
        counterLabels = {}
        for j in range(len(cluster[i])):
            counterLabels[y[cluster[i][j]]] = counterLabels.get(y[cluster[i][j]], 0) + 1
        # Assign the most frequent label to the cluster
        maxLabel = max(counterLabels, key=counterLabels.get)
        clusterLabels.append(maxLabel)
    
    pointsLabels = []
    for i in range(len(labels)):
        pointsLabels.append(clusterLabels[labels[i]])

    return cluster, pointsLabels

## Evalaution

### Load the data

In [4]:
# Loading the data
x, y = loadData('archive/kddcup.data.corrected')

### Split the data

In [5]:
# Splitting the data into train and test
x_train, x_test, y_train, y_test = splitData(x, y, train_size= 0.00025)

### Preprocessing the Data

In [6]:
# Only using the training data, which will be preprocessed
x_train = preprocessData(x_train)
print("The shape of the training data is: ", x_train.shape)

The shape of the training data is:  (1224, 69)


### Evaluating

In [7]:
clusters, y_pred = clusteringUsingNormalizedCut(x_train, 23, gamma=100, y=y_train)
y_pred = np.array(y_pred)

C:\Users\Yousef\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [11]:
total_prec = 0
total_rec = 0
f1 = 0


for i in range(len(clusters)):
    # precision is equal to the number of points in the cluster that are of the same class divided by the number of points in the cluster
    prec = 0
    # recall is equal to the number of points in the cluster that are of the same class divided by the number of points in the class
    rec = 0
    clusterLabel = y_pred[clusters[i][0]]
    numberofPointsInCluster = len(clusters[i])
    numberofPointsInClass = y_train.tolist().count(clusterLabel)
    # Getting the number of points in the cluster that are of the same class
    for j in range(len(clusters[i])):
        if y_train[clusters[i][j]] == clusterLabel:
            prec += 1
            rec += 1
    
    # Computing the precision and recall
    prec = prec / numberofPointsInCluster
    rec = rec / numberofPointsInClass
    total_prec += prec
    total_rec += rec



f1 = 2 * (total_prec/len(clusters) * total_rec/len(clusters)) / (total_prec/len(clusters) + total_rec/len(clusters))

print("The average precision(purity) is: ", total_prec/len(clusters))
print("The average recall is: ", total_rec/len(clusters))
print("The average f1 score is: ", f1)


print("-" * 50)
print("Precision: ", precision_score(y_train, y_pred, average='weighted'))
print("Recall: ", recall_score(y_train, y_pred, average='weighted'))
print("F1 score: ", f1_score(y_train, y_pred, average='weighted'))
print("Accuracy: ", accuracy_score(y_train, y_pred))


The average precision(purity) is:  0.976090634885603
The average recall is:  0.08255914777653907
The average f1 score is:  0.15224149154628283
--------------------------------------------------
Precision:  0.20741989881956155
Recall:  0.2712657712657713
F1 score:  0.22418123196802187
Accuracy:  0.7344771241830066


C:\Users\Yousef\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
